In [1]:
import pandas as pd
import warnings
warnings.filterwarnings('ignore')

from Modules.dataEngineering import dataEngineering
dataEngineering = dataEngineering()

ModuleNotFoundError: No module named 'Modules'

คำย่อที่ควรรู้ก่อน
f คือ filter ,
a คือ no filter หรือ ใช้ raw ratings ,
n คือ no ,

Load ข้อมูล

In [2]:
df_f_ratings   = dataEngineering.load_Filter_Ratings()['data']
df_a_ratings        = dataEngineering.loadRatings()['data']
df_movies           = dataEngineering.loadMovies()['data']
df_users            = dataEngineering.loadUsers()['data']

ทำให้แน่ใจว่า User_Index และ Movie_Index เป็น int

In [3]:
df_f_ratings['User_Index'] = df_f_ratings['User_Index'].astype(int)
df_f_ratings['Movie_Index'] = df_f_ratings['Movie_Index'].astype(int)

In [4]:
df_a_ratings.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100836 entries, 0 to 100835
Data columns (total 6 columns):
 #   Column      Non-Null Count   Dtype  
---  ------      --------------   -----  
 0   UserID      100836 non-null  int64  
 1   MovieID     100836 non-null  int64  
 2   userRating  100836 non-null  float64
 3   Title       100836 non-null  object 
 4   Genres      100836 non-null  object 
 5   meanRating  100836 non-null  float64
dtypes: float64(2), int64(2), object(2)
memory usage: 4.6+ MB


In [5]:
print('Count Movies  : ',len(df_a_ratings['MovieID'].unique()))
print('Count Users   : ',len(df_a_ratings['UserID'].unique()))
print('Count Ratings : ',len(df_a_ratings))

Count Movies  :  9724
Count Users   :  610
Count Ratings :  100836


สร้าง pivot table

In [6]:
df_f_index     = df_f_ratings.pivot(index='User_Index', columns='Movie_Index' , values='userRating')
df_fn_index = df_f_ratings.pivot(index='UserID', columns='MovieID' , values='userRating')

In [7]:
df_a_ratings["User_Index"] = df_a_ratings["UserID"].astype('category').cat.codes
df_a_ratings["Movie_Index"] = df_a_ratings["MovieID"].astype('category').cat.codes

In [8]:
df_a_index     = df_a_ratings.pivot(index='User_Index', columns='Movie_Index' , values='userRating')
df_an_index = df_a_ratings.pivot(index='UserID', columns='MovieID' , values='userRating')

หา learning_rate และ user_based ไหนดีที่สุด

In [9]:
# from surprise.model_selection import GridSearchCV

# sim_options = {
#     'name': ['cosine', 'pearson'],
#     'user_based': [False, True],
# }
# bsl_options = {
#     'method': ['sgd'],
#     'learning_rate': [0.0005, 0.005]
# }
# param_grid = {
#     'sim_options': sim_options,
#     'bsl_options': bsl_options
# }

# gs = GridSearchCV(KNNWithMeans, param_grid, measures=['rmse', 'mae'], cv=3)
# gs.fit(data)

In [10]:
# gs.best_params

ผลลัพธ์

In [11]:
options = {
    'bsl_options': {'learning_rate': 0.0005, 'method': 'sgd'},
    'sim_options': {'name': 'cosine', 'user_based': True}
}

In [12]:
from surprise import Dataset, Reader
from surprise.model_selection import train_test_split
from surprise import KNNWithMeans
from surprise import accuracy
from surprise.model_selection import cross_validate

In [13]:
reader = Reader(rating_scale=(1, 5))

In [14]:
def create_KNNmodel(df, k_group):
    df = df.copy()
    # print('k_group: ', k_group)
    data = Dataset.load_from_df(df.stack().reset_index(), reader)
    train_set, test_set = train_test_split(data, test_size=0.2, random_state=42)

    model = KNNWithMeans( k=k_group,
        sim_options=options['sim_options'], 
        bsl_options=options['bsl_options']
    )
    model.fit(train_set)

    # predictions = model.test(test_set)
    # rmse = accuracy.rmse(predictions)
    # mae = accuracy.mae(predictions)
    return model

In [16]:
model_f_index = create_KNNmodel(df_f_index, 5)
model_fn_index = create_KNNmodel(df_fn_index, 5)

model_a_index = create_KNNmodel(df_a_index, 5)
model_an_index = create_KNNmodel(df_an_index, 5)

Computing the cosine similarity matrix...
Done computing similarity matrix.
Computing the cosine similarity matrix...
Done computing similarity matrix.
Computing the cosine similarity matrix...
Done computing similarity matrix.
Computing the cosine similarity matrix...
Done computing similarity matrix.


ทดลองหา Models ที่ดีระหว่าง 4 Models ข้างต้น

In [17]:
def predict_model_n_index(matrix, model):
    matrix = matrix.copy()
    u_col = 0
    for u_id, row in matrix.iterrows():
        for m_col, item in enumerate(row):
            m_id = matrix.columns[m_col]
            if pd.isnull(item):
                prep_rating = round(model.predict(u_id, m_id).est, 4)
                matrix.iloc[u_col][m_id] = prep_rating
        u_col += 1
    return matrix

def predict_model_index(matrix, model):
    matrix = matrix.copy()
    for u_row, row in matrix.iterrows():
        for m_col, item in enumerate(row):
            if pd.isnull(item):
                prep_rating = round(model.predict(u_row, m_col).est, 4)
                matrix.iloc[u_row][m_col] = prep_rating
    return matrix

pivot_a_test = df_a_index.iloc[:2].copy()
pivot_an_test = df_an_index.iloc[:2].copy()


In [18]:
def predict_model_n_index(matrix, model):
    matrix = matrix.copy()
    u_col = 0
    ratings = []
    for u_id, row in matrix.iterrows():
        for m_col, item in enumerate(row):
            m_id = matrix.columns[m_col]
            if pd.isnull(item):
                prep_rating = round(model.predict(u_id, m_id).est, 4)
                ratings.append([u_id, m_id, prep_rating])
                matrix.iloc[u_col][m_id] = prep_rating
        u_col += 1
    
    recomment_unwatched = pd.DataFrame(ratings, columns=['UserID', 'MovieID', 'prep_Rating'])
    return matrix, recomment_unwatched.sort_values(by=['prep_Rating'], ascending=False)

def predict_model_index(matrix, model):
    matrix = matrix.copy()
    ratings = []
    for u_row, row in matrix.iterrows():
        for m_col, item in enumerate(row):
            if pd.isnull(item):
                prep_rating = round(model.predict(u_row, m_col).est, 4)
                matrix.iloc[u_row][m_col] = prep_rating
                ratings.append([u_row, m_col, prep_rating])

    recomment_unwatched = pd.DataFrame(ratings, columns=['User_Index', 'Movie_Index', 'prep_Rating'])
    return matrix, recomment_unwatched.sort_values(by=['prep_Rating'], ascending=False)

userID = 2

pivot_a_test  =  df_a_index.iloc[:userID].copy()
pivot_an_test = df_an_index.iloc[:userID].copy()

Model 1 แย่ที่สุด เพราะไม่สามารถทำนาย User ที่ไม่เคยเห็นได้

Model 2 กับ 4 ได้ผลเหมือนกัน

ถ้าไม่มี Index จะใช้งานได้ง่ายกว่า แบบมี Index เพราะแบบมี Indexหลังทำนายแล้วจำเป็นต้องแปลงIndex เป็น ID กลับคืนด้วย
แต่มีแบบมี Index เร็วกว่าแบบไม่มี Index

Save Model

In [19]:
import joblib

joblib.dump(model_a_index, 'models/knn_model.pkl')

['models/knn_model.pkl']

In [20]:
model = joblib.load('models/knn_model.pkl')

In [21]:
def predict_only_movies_unwatched(matrix):
    matrix = matrix.copy()
    ratings = []
    
    for u_index, row in matrix.iterrows():
        for m_col, item in enumerate(row):
            if pd.isnull(item):
                prep_rating = round(model_f_index.predict(u_index, m_col).est, 4)
                ratings.append([u_index, m_col, prep_rating])
            else:
    # ทำให้หนังที่เคยดูแล้วไม่ขึ้นในการแนะนำหนัง
                ratings.append([u_index, m_col, 0])

    recomment_unwatched = pd.DataFrame(ratings, columns=['User_Index', 'Movie_Index', 'prep_Rating'])
    return recomment_unwatched

In [22]:
def recommendTop10Movie(userID, df_ratings ,top_n=10):
    df_ratings = df_ratings.copy()
    # Find User_Index in df_ratings
    try:
        userIndex = df_ratings[df_ratings['UserID'] == userID].User_Index.unique()[0]
        df_f_index = df_ratings.pivot(index='UserID', columns='MovieID' , values='userRating')

        matrix = df_f_index[df_f_index.index == userIndex]
        
        prep_movies = predict_model_n_index(matrix, model_fn_index)[1]

        return prep_movies.sort_values(by=['prep_Rating'], ascending=False).iloc[:top_n]
    except IndexError:
        txt = (f'UserID: {userID} not in database')
        return txt
# prep_movies.sort_values(by=['prep_Rating'], ascending=False).iloc[0:top_n]
recommendTop10Movie(userID = 2, df_ratings = df_a_ratings)

,UserID,MovieID,prep_Rating
7562,1,92494,5.0
3513,1,5224,5.0
3520,1,5240,5.0
8844,1,143511,5.0
8846,1,143559,5.0
8881,1,146684,5.0
3185,1,4658,5.0
8879,1,146662,5.0
9353,1,176329,5.0
9368,1,177593,5.0
